In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

In [ ]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.sphere, [.03])
RealWorld.getFrame("obj0").setPosition([0., .2, 2.])
RealWorld.getFrame("obj0").setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
#     print("deleting", name)
    RealWorld.delFrame(name)
    
    
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [ ]:
# setup your model world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

obj = C.addFrame("object")
obj.setColor([1.,0,0]) # set the color of one objet to red!
obj.setShape(ry.ST.sphere, [.03]) # assume that we know the object shape

D = C.view();

In [ ]:
points = []
tau = .01

gripping = False
grasped = False

for t in range(3000):
    time.sleep(tau)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    
    # TOTAL CHEAT: grab the true position from the RealWorld
    objectPosition = RealWorld.frame("obj0").getPosition();
    obj.setPosition(objectPosition) # set the model object to percept
    
    C.setJointState(q); # set your robot model to match the real q
    
    y, J = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])
    
    qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)
    
    if not gripping and np.linalg.norm(y) < .001:
        S.closeGripper("R_gripper", speed=1.)
        gripping = True
        
    if gripping and S.getGripperWidth("R_gripper") < 1e-2:
        print("FAILED!")
        S.openGripper("R_gripper", speed=1.)
        gripping = False
        
    if gripping and S.getGripperIsGrasping("R_gripper"):
        print("GRASPED!")
        grasped = True
        break

    S.step(qdot, tau, ry.ControlMode.velocity)